In [1]:
import pandas as pd  
import numpy as np
import re
import string

In [11]:
df = pd.read_csv('C:/KULIAH/SEMESTER 7/skripsi bismillah fix/SHERINA AVIANITA/svm/layanan/aspek_layanan2.csv')

In [12]:
del df['No']
df.head(10)

,Reviewer Name,Review,Label
0,wong tengah,Ribet pemesanannya,-1
1,Saiful Rijal,tidak masuk pelayanannya,-1
2,Fitri Ayu Meriana,pelayanan cepat,1
3,Toni Dimas A.,lama pelayanan beli nasi sayur akhirnya aku ti...,-1
4,Alex Budiman,Pelayanan Cepat & Kasir untuk Ojek Online dise...,1
5,Erasmus Reyhan I,Untuk parkiran baik motor atau mobil cukup lua...,1
6,Saka Tara,"Enak buat nongkrong, selalu konsisten pelayana...",1
7,pyu chan,Nongkrong sekaligus kuliner. Pesan dulu makana...,0
8,suharmono simon,Kurang memuaskan pelayanan dan makanannya,-1
9,Eko Rahmad Hermansah,pelayanan baik,1


## Preprocessing Data

In [13]:
#case folding 

def caseFolding(textReview):

 textReview = textReview.lower()
 return(textReview)

# apply to each datas and insert them into new column
df['case folding'] = df['Review'].apply(lambda x:
caseFolding(x))


In [14]:
#Remove Punctuation

def cleansing(textReview):
 # remove numbering
 textReview = [re.sub('[0-9]', '', i) for i in textReview]
 # remove punct
 textReview = "".join([c for c in textReview if c not in
string.punctuation])
 # remove enter
 textReview = textReview.replace('\n',' ')
 return textReview

# apply to each datas and insert them into new column
df['cleansing'] = df['case folding'].apply(lambda x: cleansing(x))

In [15]:
#Tokenization

from nltk.tokenize import RegexpTokenizer

# To detect a word character (a-z, A-Z, 0-9, _)
wordTokenize = RegexpTokenizer(r'\w+')

# Function to detect empty string
def removeEmptyToken(textReview):
 textReview = [x for x in textReview if len(x) > 0]
 return textReview

# apply to each datas and insert them into new column
df['tokenizing'] = df['cleansing'].apply(lambda x:
wordTokenize.tokenize(x))
df['tokenizing'] = df['tokenizing'].apply(lambda x:
removeEmptyToken(x))

In [16]:
#Stopword Removal

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover,ArrayDictionary 

# stopword removal
factory_stopword = StopWordRemoverFactory().get_stop_words()
exclude_from_stopword= ['tidak', 'namun', 'sementara', 'hanya', 'sedangkan', 'tetapi', 'kecuali', 'agak']
remove_word = ([word for word in factory_stopword if word not in exclude_from_stopword])
add_word=['sih','loh','kok','aja','deh','lah','nah', 'yg', 'kalau', 'lalu', 'nya', 'sini', 'sangat', 'banget']
data= remove_word + add_word
final_dictionary= ArrayDictionary(data)

def stopwords(textReview):
    stopword_remover = StopWordRemover(final_dictionary)
    textReview = [stopword_remover.remove(x)for x in textReview]
    return(textReview)

df['stopwords'] = df['tokenizing'].apply(lambda x:stopwords(x))


In [17]:
#Stemming

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from Sastrawi.Dictionary.ArrayDictionary import ArrayDictionary
from Sastrawi.StopWordRemover.StopWordRemover import StopWordRemover
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Stemming function to change all affixed words into basic tenses
def word_stemmer(textReview):
    factorystemmer = StemmerFactory()
    stemmer = factorystemmer.create_stemmer()
    textReview = [stemmer.stem(x)for x in textReview]
    return(textReview)

# apply to each datas and insert them into new column
df['stemming'] = df['stopwords'].apply(lambda x:word_stemmer(x))

# Function to detect empty token
def removeEmptyToken(textReview):
    textReview = [x for x in textReview if len(x) > 0]
    return textReview

# apply to each datas and insert them into new column
df['final preprocessing'] = df['stemming'].apply(lambda x:
removeEmptyToken(x))


In [18]:
#Menyimpan Hasil Preprocessing

df.to_csv('C:/KULIAH/SEMESTER 7/skripsi bismillah fix/SHERINA AVIANITA/svm/layanan/skripsi/preprocessing_test_copy.csv')

In [19]:
df2 = pd.read_csv('C:/KULIAH/SEMESTER 7/skripsi bismillah fix/SHERINA AVIANITA/svm/layanan/skripsi/preprocessing_test_copy.csv', index_col=0)
df2.head()

,Reviewer Name,Review,Label,case folding,cleansing,tokenizing,stopwords,stemming,final preprocessing
0,wong tengah,Ribet pemesanannya,-1,ribet pemesanannya,ribet pemesanannya,"['ribet', 'pemesanannya']","['ribet', 'pemesanannya']","['ribet', 'mesan']","['ribet', 'mesan']"
1,Saiful Rijal,tidak masuk pelayanannya,-1,tidak masuk pelayanannya,tidak masuk pelayanannya,"['tidak', 'masuk', 'pelayanannya']","['tidak', 'masuk', 'pelayanannya']","['tidak', 'masuk', 'layan']","['tidak', 'masuk', 'layan']"
2,Fitri Ayu Meriana,pelayanan cepat,1,pelayanan cepat,pelayanan cepat,"['pelayanan', 'cepat']","['pelayanan', 'cepat']","['layan', 'cepat']","['layan', 'cepat']"
3,Toni Dimas A.,lama pelayanan beli nasi sayur akhirnya aku ti...,-1,lama pelayanan beli nasi sayur akhirnya aku ti...,lama pelayanan beli nasi sayur akhirnya aku ti...,"['lama', 'pelayanan', 'beli', 'nasi', 'sayur',...","['lama', 'pelayanan', 'beli', 'nasi', 'sayur',...","['lama', 'layan', 'beli', 'nasi', 'sayur', 'ak...","['lama', 'layan', 'beli', 'nasi', 'sayur', 'ak..."
4,Alex Budiman,Pelayanan Cepat & Kasir untuk Ojek Online dise...,1,pelayanan cepat & kasir untuk ojek online dise...,pelayanan cepat kasir untuk ojek online disen...,"['pelayanan', 'cepat', 'kasir', 'untuk', 'ojek...","['pelayanan', 'cepat', 'kasir', '', 'ojek', 'o...","['layan', 'cepat', 'kasir', '', 'ojek', 'onlin...","['layan', 'cepat', 'kasir', 'ojek', 'online', ..."


In [12]:
#Term Weighting TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_weighting = TfidfVectorizer()

# Implement the fit transformation of TF-IDF calculation (matrix)
tfidf_fit_transform = tfidf_weighting.fit_transform(df2['final preprocessing'])
                                                           
# Extract data from dataset features (terms) to column
tfidf_feature_names = tfidf_weighting.get_feature_names()

# Return a dense matrix representation of a matrix NDFrame
tfidf_dense = tfidf_fit_transform.todense()

# Convert a series into list
tfidf_denselist = tfidf_dense.tolist()

# Save the result of weighting to a DataFrame
df_tfidf_final = pd.DataFrame(tfidf_denselist, columns=tfidf_feature_names)

# Export the results (DataFrame) to Excel file
df_tfidf_final.to_csv('C:/KULIAH/SEMESTER 7/skripsi bismillah fix/SHERINA AVIANITA/svm/layanan/skripsi/tfidf_test_copy.csv', encoding='utf-8')

df3 = pd.read_csv('C:/KULIAH/SEMESTER 7/skripsi bismillah fix/SHERINA AVIANITA/svm/layanan/skripsi/tfidf_test_copy.csv', index_col=0)
df3.head()


,abis,ac,acung,ada,adain,aduh,again,agak,agam,agin,...,waktu,wanita,waras,wib,wifi,wkwk,worth,yaa,yups,yupskuliner
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Klasifikasi SVM

In [13]:
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

In [54]:
df_aspek = pd.read_csv('C:/KULIAH/SEMESTER 7/skripsi bismillah fix/SHERINA AVIANITA/svm/layanan/skripsi/preprocessing_test_copy.csv', index_col=0)
df_tw = pd.read_csv('C:/KULIAH/SEMESTER 7/skripsi bismillah fix/SHERINA AVIANITA/svm/layanan/skripsi/tfidf_test_copy.csv')

X = np.array(df_tw.values.tolist())
y = np.array(df_aspek['Label'].values.tolist()) 


acc = []
prediksi = []
ekspektasi = []

classifier = LinearSVC(multi_class='ovr',random_state=0 )
n_iterasi = 1000
for i in range (n_iterasi):
    skf = StratifiedKFold (n_splits=10, random_state=0, shuffle=True)
    skf.get_n_splits(X, y)
x = 1

writer = pd.ExcelWriter('C:/KULIAH/SEMESTER 7/skripsi bismillah fix/SHERINA AVIANITA/svm/layanan/skripsi/prediksi_layanan_copy.xlsx')

#partisi data
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    classifier.fit(X_train,y_train) 

#menyimpan data prediksi
    prediction= classifier.predict(X_test)
    prediksi.extend(prediction)
    excelDF = []
    for i, val in enumerate(test_index):
        excelDF.append(pd.DataFrame({
            "INDEX": [val],
            "ULASAN": [df_aspek['Review'][val]],
            "LABEL_ASLI": [df_aspek['Label'][val]],
            "LABEL_PREDIKSI": [prediction[i]]
        }))
    
    excelDF = pd.concat(excelDF)
    excelDF.to_excel(writer, sheet_name="Fold-"+str(x))
    
    # menyimpan data label yang diharapkan
    ekspektasi.extend(y_test)
    
    #menghitung akurasi
    score=accuracy_score(y_test,prediction)
    acc.append(score)
    x += 1
    writer.save()
    


c:\users\sherin\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\svm\_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
c:\users\sherin\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\svm\_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
c:\users\sherin\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\svm\_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
c:\users\sherin\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\svm\_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
c:\users\sherin\appd

In [55]:
#evaluasi confusion matrix

accuracy = np.array(acc)
print(accuracy)

confusionmatrix = confusion_matrix(ekspektasi,prediksi)
print(confusionmatrix)

report=metrics.classification_report(ekspektasi,prediksi)
print(report)


[0.83870968 0.87096774 0.80645161 0.80645161 0.93548387 0.70967742
 0.87096774 0.83333333 0.96666667 0.96666667]
[[123   8   4]
 [  9  27   7]
 [ 13   2 114]]
              precision    recall  f1-score   support

          -1       0.85      0.91      0.88       135
           0       0.73      0.63      0.68        43
           1       0.91      0.88      0.90       129

    accuracy                           0.86       307
   macro avg       0.83      0.81      0.82       307
weighted avg       0.86      0.86      0.86       307



In [56]:
#evaluasi confusion matrix
accuracy = np.array(acc).mean()
print('Akurasi Rata-Rata:', accuracy)

Akurasi Rata-Rata: 0.8605376344086022


In [62]:
#evaluasi confusion matrix

accuracy = np.array(acc)
print('Akurasi Tiap Fold:', accuracy)
accuracy = np.array(acc).mean()
print('Akurasi Rata-Rata:', accuracy)

confusionmatrix = confusion_matrix(ekspektasi,prediksi)
print('\n', confusionmatrix)
report=metrics.classification_report(ekspektasi,prediksi)
print('\n', report)


Akurasi Tiap Fold: [0.83870968 0.87096774 0.80645161 0.80645161 0.93548387 0.70967742
 0.87096774 0.83333333 0.96666667 0.96666667]
Akurasi Rata-Rata: 0.8605376344086022

 [[123   8   4]
 [  9  27   7]
 [ 13   2 114]]

               precision    recall  f1-score   support

          -1       0.85      0.91      0.88       135
           0       0.73      0.63      0.68        43
           1       0.91      0.88      0.90       129

    accuracy                           0.86       307
   macro avg       0.83      0.81      0.82       307
weighted avg       0.86      0.86      0.86       307



# Kata Teratas

In [20]:
import ast

def join_text_list(texts):
    texts = ast.literal_eval(texts)
    return ' '.join([text for text in texts])
df2["text_join"] = df2["final preprocessing"].apply(join_text_list)

df2["text_join"].head()

0                                     ribet mesan
1                               tidak masuk layan
2                                     layan cepat
3    lama layan beli nasi sayur akhir aku tinggal
4     layan cepat kasir ojek online sendiri kasir
Name: text_join, dtype: object

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

cvector = CountVectorizer(min_df = 0.0, max_df = 1.0, ngram_range=(1,2))
cvector.fit(df2.text_join)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=0.0,
                ngram_range=(1, 2), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [26]:
#Positif

pos = cvector.transform(df2[df2.Label == 1].text_join)

pos_words = pos.sum(axis=0)
pos_words_freq = [(word, pos_words[0, idx]) for word, idx in cvector.vocabulary_.items()]
pos_tf = pd.DataFrame(list(sorted(pos_words_freq, key = lambda x: x[1], reverse=True)),columns=['Terms','positif'])

pos_tf_df = pos_tf.set_index('Terms')
pos_tf_df.head(15)

,positif
Terms,
layan,80
makan,45
tempat,45
cepat,41
layan cepat,24
banyak,23
baik,22
nyaman,21
enak,21


In [27]:
#Negatif

neg = cvector.transform(df2[df2.Label == -1].text_join)

neg_words = neg.sum(axis=0)
neg_words_freq = [(word, neg_words[0, idx]) for word, idx in cvector.vocabulary_.items()]
neg_tf = pd.DataFrame(list(sorted(neg_words_freq, key = lambda x: x[1], reverse=True)),columns=['Terms','count'])

neg_tf_df = neg_tf.set_index('Terms')
neg_tf_df.head(15)

,count
Terms,
makan,46
tidak,45
layan,42
kasir,39
antre,31
lama,29
tempat,26
pesan,25
bayar,24


In [25]:
#Netral

net = cvector.transform(df2[df2.Label == 0].text_join)

net_words = net.sum(axis=0)
net_words_freq = [(word, net_words[0, idx]) for word, idx in cvector.vocabulary_.items()]
net_tf = pd.DataFrame(list(sorted(net_words_freq, key = lambda x: x[1], reverse=True)),columns=['Terms','count'])

net_tf_df = net_tf.set_index('Terms')
net_tf_df.head(10)

,count
Terms,
makan,51
bayar,33
kasir,30
tempat,27
pesan,25
kartu,14
bayar kasir,12
duduk,11
menu,9
